In [2]:
import os
import pandas as pd

#### create path to my data on pc

In [4]:
train_path = r"C:\Users\1\Desktop\University_semeter_3\ZNEUS\archive\ASL_Dataset\Train"

#### create csv file with image pases and labels

In [10]:
data = []

for folder in os.listdir(train_path):
    folder_path = os.path.join(train_path, folder)

    if os.path.isdir(folder_path):
        # Проходимося по всіх фото в папці
        for image in os.listdir(folder_path):
            if image.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(folder_path, image)
                data.append({'path': image_path, 'label': folder})

# Створюємо DataFrame
df = pd.DataFrame(data)

# Зберігаємо в CSV
df.to_csv('dataset.csv', index=False)
print(f"✅ Готово! Створено {len(df)} рядків")
print(f"📊 Класи: {df['label'].unique()}")


✅ Готово! Створено 165670 рядків
📊 Класи: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'Nothing' 'O' 'P'
 'Q' 'R' 'S' 'Space' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [11]:
ds = pd.read_csv('dataset.csv')
ds.head()


,path,label
0,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...,A
1,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...,A
2,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...,A
3,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...,A
4,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...,A


In [12]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165670 entries, 0 to 165669
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   path    165670 non-null  object
 1   label   165670 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [13]:
ds.duplicated().sum()

np.int64(0)